In [31]:
import pandas as pd 
from dotenv import load_dotenv
import os
import numpy as np

import mlflow
from datetime import date

In [23]:
# get environment variables
load_dotenv(dotenv_path="../.env")
DATAPATH = os.getenv("DATAPATH")
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")

# Set other variables
model_name = "best-model-2023-08-03"
preprocessor_name = "preprocessor-2023-08-03"
stage = "Production"

In [19]:
data = pd.read_pickle(f"{DATAPATH}/BEL_20.pkl").sort_values("Date") # Load data

# Find the maximum date in the DataFrame
max_date = data.index.max()

# Filter the DataFrame to get only the rows with the maximum date
data = data[data.index == max_date]

In [20]:
# Rename columns by shifting names one place to the left
old_col_names = data.filter(like="close_growth").columns.tolist()
last_col_name = old_col_names[-1] # get last element
old_col_names.pop() # remove last element from list

new_col_names = data.filter(like="close_growth_lag").columns.tolist()

data = data.drop(columns=last_col_name) # drop last column

In [22]:
# Using list comprehension to create the dictionary
mapping = {old_col_names[i]: new_col_names[i] for i in range(len(old_col_names))}

# Rename the columns using the dictionary
data.rename(columns=mapping, inplace=True)

In [25]:
# Load model
preprocessor = mlflow.sklearn.load_model(model_uri=f"models:/{preprocessor_name}/{stage}")
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{stage}")

In [43]:
# Make predictions
data_reduced = preprocessor.transform(data)
predictions = model.predict(data_reduced).tolist()

# Give advice based on predictions
advice = pd.DataFrame({"ticker": data["ticker"], "prediction": predictions})
advice["advice"] = advice["prediction"].apply(lambda x: "BUY" if x[0] > 0 else "SELL")
advice.reset_index(drop=True, inplace=True)


/Users/timcosemans/.local/share/virtualenvs/PROJ-Algorithmic-Trading-enWZjsr5/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [44]:
# Write predictions to json file
advice.set_index("ticker", inplace=False).to_json(f'{DATAPATH}/advice.json')